<center><h5>OPEN DATA PROJECT</h5></center>


<center><h1>THEFT IN RESIDENTIAL DEVELOPMENTS</h1></center>

<center><h1>-</h1></center>

<center><small>ELIJAH ESMERO</small></center>

<center><img src="images/land_value_data_webpage.png"></center>

<center><img src="images/crime_data_webpage.png"></center>

<center><img src="images/raw_sydney_map.png"></center>

<center><img src="images/edited_sydney_map.png"></center>

<center><img src="images/marsdenpark_annual_progression/map_20130816.png"></center>

<center><img src="images/marsdenpark_annual_progression/map_20190719.png"></center>

<center><img src="images/marsdenpark_theft_facebookposts/theft collage.png"></center>

<center><h1>DO RESIDENTIAL DEVELOPMENTS EXPERIENCE MORE THEFT?</h1></center>

<center><h1>DOES THE "THEFT PER HOUSE" IN A SUBURB INCREASE AS THE NUMBER OF HOUSES INCREASE?</h1></center>

<center><h1>TIME SERIES</h1></center>

In [1]:
# ==============================
# DOWNLOADING LAND VALUE DATASET
# ==============================

required_zipfiles_list = [
 'LV_20170701.zip',
 'LV_20170801.zip',
 'LV_20170901.zip',
 'LV_20171001.zip',
 'LV_20171101.zip',
 'LV_20171201.zip',
 'LV_20180101.zip',
 'LV_20180201.zip',
 'LV_20180301.zip',
 'LV_20180401.zip',
 'LV_20180501.zip',
 'LV_20180601.zip',
 'LV_20180701.zip',
 'LV_20180801.zip',
 'LV_20180901.zip',
 'LV_20181001.zip',
 'LV_20181101.zip',
 'LV_20181201.zip',
 'LV_20190101.zip',
 'LV_20190201.zip',
 'LV_20190301.zip',
 'LV_20190401.zip',
 'LV_20190501.zip',
 'LV_20190601.zip',
 'LV_20190701.zip']

In [2]:
# ==============================
# DOWNLOADING LAND VALUE DATASET
# ==============================

import requests
from pathlib import Path

# make list of local zip file paths
local_zipfile_path_list = sorted(Path("data/land_value_data").glob("**/*.zip"))

# make a list of strings from the local zip file paths
local_zipfile_list = [path.name for path in local_zipfile_path_list]

# check if required zipfiles are already downloaded, if not then download them
while(required_zipfiles_list != local_zipfile_list): # to repeat loop as case r.status_code may not be 200
    for zipfile in required_zipfiles_list:
        if zipfile in local_zipfile_list:
            print(zipfile + " is already downloaded")
        else: 
            url = "https://www.valuergeneral.nsw.gov.au/land_value_summaries/lvfiles/{}".format(zipfile)

            # get filename from url
            filename = url.split("/")[-1]

            # download file from url
            r = requests.get(url, stream = True)
            if r.status_code is 200:
                print("downloading " + zipfile)
                with open(filename,"wb") as f:
                    f.write(r.content)
            
# revise paths of local zip files
local_zipfile_path_list = sorted(Path("data/land_value_data").glob("**/*.zip"))

<center><img src="images/zipfile_extraction_size_difference.png"></center>

<center><img src="images/zipfile_contents.png"></center>

In [43]:
# ============================
# ACCESSING LAND VALUE DATASET
# ============================

dtype_dict={"DISTRICT CODE": int, 
            "DISTRICT NAME": str,
            "PROPERTY ID": int,
            "PROPERTY TYPE": str,
            "PROPERTY NAME": str,
            "UNIT NUMBER": str,
            "HOUSE NUMBER": str,
            "STREET NAME": str,
            "SUBURB NAME": str,
            "POSTCODE": float,
            "PROPERTY DESCRIPTION": str,
            "ZONE CODE": str,
            "AREA": float,
            "AREA TYPE": str,
            "BASE DATE 1": str,
            "LAND VALUE 1": float,
            "AUTHORITY 1": str,
            "BASIS 1": str,
            "BASE DATE 2": str,
            "LAND VALUE 2": float,
            "AUTHORITY 2": str,
            "BASIS 2": str,
            "BASE DATE 3": str,
            "LAND VALUE 3": float,
            "AUTHORITY 3": str,
            "BASIS 3": str,
            "BASE DATE 4": str,
            "LAND VALUE 4": float,
            "AUTHORITY 4": str,
            "BASIS 4": str,
            "BASE DATE 5": str,
            "LAND VALUE 5": float,
            "AUTHORITY 5": str,
            "BASIS 5": str,
            "Unnamed: 34": str}

In [69]:
# ============================
# ACCESSING LAND VALUE DATASET
# ============================

def concat_all_csv_in_zipfile_into_one_df(zipfile_path):
    with zipfile.ZipFile(zipfile_path) as zip:
        csv_in_zipfile_list = sorted(zip.namelist())[:-1]  # to exclude the commons.txt file
        df = pd.concat([pd.read_csv(zip.open(csv),
                                    encoding="ISO-8859-1",
                                    error_bad_lines=False,
                                    dtype=dtype_dict) for csv in csv_in_zipfile_list], sort=False)
    return df

concat_all_csv_in_zipfile_into_one_df(local_zipfile_path_list[0])

,DISTRICT CODE,DISTRICT NAME,PROPERTY ID,PROPERTY TYPE,PROPERTY NAME,UNIT NUMBER,HOUSE NUMBER,STREET NAME,SUBURB NAME,POSTCODE,...,BASIS 3,BASE DATE 4,LAND VALUE 4,AUTHORITY 4,BASIS 4,BASE DATE 5,LAND VALUE 5,AUTHORITY 5,BASIS 5,Unnamed: 34
0,1,CESSNOCK,3017030,NORMAL,NaN,NaN,25,ECHIDNA CL,BELLBIRD,2325.0,...,6A(1),01/07/2013,251000.0,14A(1),6A(1),01/07/2012,239000.0,14A(1),6A(1),NaN
1,1,CESSNOCK,3017031,NORMAL,NaN,NaN,27,ECHIDNA CL,BELLBIRD,2325.0,...,6A(1),01/07/2013,251000.0,14A(1),6A(1),01/07/2012,239000.0,14A(1),6A(1),NaN
2,1,CESSNOCK,3017032,NORMAL,NaN,NaN,41,ECHIDNA CL,BELLBIRD,2325.0,...,6A(1),01/07/2013,251000.0,14A(1),6A(1),01/07/2012,239000.0,14A(1),6A(1),NaN
3,1,CESSNOCK,3017033,NORMAL,NaN,NaN,43,ECHIDNA CL,BELLBIRD,2325.0,...,6A(1),01/07/2013,251000.0,14A(1),6A(1),01/07/2012,239000.0,14A(1),6A(1),NaN
4,1,CESSNOCK,3017034,NORMAL,NaN,NaN,45,ECHIDNA CL,BELLBIRD,2325.0,...,6A(1),01/07/2013,242000.0,14A(1),6A(1),01/07/2012,230000.0,14A(1),6A(1),NaN
5,1,CESSNOCK,3017035,NORMAL,NaN,NaN,47,ECHIDNA CL,BELLBIRD,2325.0,...,6A(1),01/07/2013,242000.0,14A(1),6A(1),01/07/2012,230000.0,14A(1),6A(1),NaN
6,1,CESSNOCK,3017036,NORMAL,NaN,NaN,49,ECHIDNA CL,BELLBIRD,2325.0,...,6A(1),01/07/2013,242000.0,14A(1),6A(1),01/07/2012,230000.0,14A(1),6A(1),NaN
7,1,CESSNOCK,3017037,NORMAL,NaN,NaN,51,ECHIDNA CL,BELLBIRD,2325.0,...,6A(1),01/07/2013,236000.0,14A(1),6A(1),01/07/2012,225000.0,14A(1),6A(1),NaN
8,1,CESSNOCK,3017038,NORMAL,NaN,NaN,53,ECHIDNA CL,BELLBIRD,2325.0,...,6A(1),01/07/2013,236000.0,14A(1),6A(1),01/07/2012,225000.0,14A(1),6A(1),NaN
9,1,CESSNOCK,3017019,NORMAL,NaN,NaN,3,ECHIDNA CL,BELLBIRD,2325.0,...,6A(1),01/07/2013,225000.0,14A(1),6A(1),01/07/2012,214000.0,14A(1),6A(1),NaN


# DISSECTING THE QUESTION

# ACCESSING THE DATA

# UNDERSTANDING THE QUESTION

# MY PLAN OF ATTACK

# USEFUL & INTERESTING CODE

# THE RESULTS